In [1]:
import os
import pandas as pd
import numpy as np 
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
from playlist_membership_check import get_tracks_positions_in_playlists
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [2]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-10.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [3]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [4]:
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialize lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that we have all the data, put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame 
popularity_df.head()


,artist_name,track_name,popularity_score
0,Noah Kahan,Stick Season,98
1,Jack Harlow,Lovin On Me,99
2,Taylor Swift,Cruel Summer,99
3,Benson Boone,Beautiful Things,96
4,Tate McRae,greedy,100


In [5]:
df['popularity_score'] = popularity_df['popularity_score']

In [6]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [7]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,10-02-2024,1,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,1,128,315270,98
1,10-02-2024,2,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,3,93,276470,99
2,10-02-2024,3,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,4,371,264974,99
3,10-02-2024,4,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,2,23,262529,96
4,10-02-2024,5,Tate McRae,greedy,3rUGC1vUpkDG9CZFHMur1t,RCA Records Label,1,6,149,226430,100


Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [8]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [9]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

Need to change "=" to "0", remove "+", and handle "RE"

In [10]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "NEW" at position 46
Non-numeric values found:
      rank rank_change                                       artist_title  \
46     47         NEW            Kanye West - TALKING (w/ Ty Dolla $ign)   
149   150         NEW              Kanye West - STARS (w/ Ty Dolla $ign)   
160   161          RE  Taylor Swift - Miss Americana & The Heartbreak...   
163   164          RE            Taylor Swift - Style (Taylor's Version)   
167   168          RE                   Bruno Mars - When I Was Your Man   
175   176          RE               USHER - Yeah! (w/ Lil Jon, Ludacris)   
179   180          RE                  Arctic Monkeys - I Wanna Be Yours   
182   183          RE                         Taylor Swift - Blank Space   
186   187          RE  Taylor Swift - All Too Well (10 Minute Version...   
189   190          RE                                 Lil Tecca - 500lbs   
195   196          RE                           

/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_57538/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [11]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [12]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Noah Kahan - Stick Season,0,15.0,-22876.0,2330994,-6333,22683775
1,Jack Harlow - Lovin On Me,1,72.0,-9000.0,1952065,-21020,29625762
2,Taylor Swift - Cruel Summer,1,1.0,18308.0,1657245,31123,52638086


In [13]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [14]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [15]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [16]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,10-02-2024,Noah Kahan,Stick Season,1,1,0,1,15.0,128,98,315270,-22876.0,2330994,-6333,22683775,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
1,10-02-2024,Jack Harlow,Lovin On Me,2,3,1,1,72.0,93,99,276470,-9000.0,1952065,-21020,29625762,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
2,10-02-2024,Taylor Swift,Cruel Summer,3,4,1,1,1.0,371,99,264974,18308.0,1657245,31123,52638086,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
3,10-02-2024,Benson Boone,Beautiful Things,4,2,-2,2,3.0,23,96,262529,-29430.0,1914246,41621,4413375,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
4,10-02-2024,Tate McRae,greedy,5,6,1,1,9.0,149,100,226430,625.0,1636712,-10702,33902405,RCA Records Label,3rUGC1vUpkDG9CZFHMur1t


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          45 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  188 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [18]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.tail()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
995,09-02-2024,Queen,Don't Stop Me Now - Remastered 2011,196,-1,RE,48,NaN,550,86,48179,NaN,143888,1214,20925790,EMI,5T8EDUDqKcs6OSOwEsfqG7
996,09-02-2024,AC/DC,You Shook Me All Night Long,197,-1,RE,51,NaN,729,85,48135,NaN,97872,-683,28801014,Columbia,2SiXAy7TuUkycRVbbWDEpo
997,09-02-2024,The Weeknd,Die For You,198,191,-7,21,NaN,631,91,48079,-1498.0,344460,-118,41448709,Universal Republic Records,2LBqCSwhJGcFQeTHMVGwy3
998,09-02-2024,Jet,Are You Gonna Be My Girl,199,-1,RE,88,NaN,70,79,47925,NaN,95534,-762,3141545,BMG Rights Management (Australia) Pty Ltd.,305WCRhhS10XUcH6AEwZk6
999,09-02-2024,Taylor Swift,Lavender Haze,200,170,-30,1,1.0,304,84,47910,-3087.0,242672,47910,22840568,Taylor Swift,5jQI2r1RdgtuT8S3iG8zFC


In [19]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [20]:
print(len(combined_df))

1200


In [21]:
combined_df.to_csv('combined_data/combined.csv', index=False)

## Example usage of `get_tracks_position_in_playlist`

Example: Checking which playlists and positions Noah Kahan's 'Stick Season' is in

In [22]:
# Example: Checking playlists and position for Noah Kahan 'Stick Season'
# using it's track_id
track_ids = [
    '0mflMxspEfB0VbI1kyLiAv', 
    # ... add other track IDs
]

track_positions = get_tracks_positions_in_playlists(client_id, client_secret, redirect_uri, playlists_dict, track_ids)

for track_id, playlists in track_positions.items():
    print(f"Track ID {track_id} is in the following playlists at these positions:")
    for playlist_name, positions in playlists.items():
        positions_str = ", ".join(str(position + 1) for position in positions)  # Create a string of positions, accounting for 0 indexing
        print(f"Playlist: {playlist_name}: Position #{positions_str}")
    if not playlists:
        print("This track is not in any of the given playlists.")

Track ID 0mflMxspEfB0VbI1kyLiAv is in the following playlists at these positions:
Playlist: Hot Hits Australia: Position #5
Playlist: Today's Top Hits: Position #12
